# Code for converting an observation to solar coordinates

## Step 1: Run the pipeline on the data to get mode06 files with the correct status bit setting.

Note that as of nustardas verion 1.6.0 you can now set the "runsplitsc" keyword to automatically split the CHU combinations for mode06 into separate data files.

These files will be stored in the event_cl output directory and have filenames like: nu20201001001A06_chu2_N_cl.evt

### Optional: Check and see how much exposure is in each file.

Use the [Observation Report Notebook](Observation_Report.ipynb) example to see how to do this.

## Step 2: Convert the data to heliocentric coordinates.

Below uses the nustar.convert methods to change the image to heliocentric coordinates from RA/dec coordinates.


Load the python libraries that we're going to use:

In [1]:
import sys
from os.path import *
import os

# For loading the NuSTAR data
from astropy.io import fits

# Load the NuSTAR python libraries
import nustar_pysolar as nustar




### Get the data from the FITS file.

Here we loop over the header keywords to get the correct columns for the X/Y coordinates. We also parse the FITS header to get the data we need to project the X/Y values (which are integers from 0-->1000) into RA/dec coordinates.

In [5]:
infile = '/Users/bwgref/science/solar/july_2016/data/20201002001/event_cl/nu20201002001B06_chu3_N_cl.evt'
hdulist = fits.open(infile)
evtdata = hdulist[1].data 


hdr = hdulist[1].header
hdulist.close()

### Rotate to solar coordinates:

Variation on what we did to setup the pointing.

Note that this can take a little bit of time to run (~a minute or two).

The important optin here is how frequently one wants to recompute the position of the Sun. The default is once every 5 seconds.

Note that this can take a while (~minutes), so I recommend saving the output as a new FITS file (below).

In [6]:
(newdata, newhdr) = nustar.convert.to_solar(evtdata, hdr, maxEvt=100)

### Write the output to a new FITS file.

Below keeps the RAWX, RAWY, DET_ID, GRADE, and PI columns from the original file. It repalces the X/Y columns with the new sun_x, sun_y columns.

In [5]:
# # Make the new filename:
(sfile, ext)=splitext(infile)

outfile=sfile+'_sunpos.evt'

# Remove output file if necessary
if isfile(outfile):
    print(outfile, 'exists! Removing old version...')
    os.remove(outfile)

fits.writeto(outfile, newdata, newhdr)

../data/Sol_16208/20201001001/event_cl/nu20201001001B06_chu3_N_cl_sunpos.evt exists! Removing old version...
